# Lab 2: Naive Bayes

In this lab we'll be training and evaluating a Naive Bayes's classifier on our two triangles datasets to predict whether a participant is depressed/scizophrenic. 

First off, let's start with some imports, as usual.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier

Now, load the datasets from Lab1 and save them as ```depression_data``` and ```scizophreania_data```. Display the first 5 rows of each dataset, to see if everything looks ok.

In [ ]:
# Code here.


## 1. sklearn and the object-oriented framework

Before we get started, now is maybe a good time for a quick introduction to object-oriented programming, and how it is applied in the ```sklearn``` library. ```scikit-learn```, or ```sklearn``` for short is a machine learning toolkit containing many useful things, ranging from classifiers such as Naive Bayes or Logistic Regression, over data transformation techniques such as PCA/dimensionality reduction, to preprocessing and evaluation modules. Stop here and take a browse through its <a href="http://scikit-learn.org/stable/documentation.html">documentation</a>, to get an overview. You do not need to understand everything, or study it front to back, but try to get a feel for what kinds of stuff it contains, and how the programs using it are typically structured.

Like many python libraries (one you might be quite familiat with is ```psychopy```), sklearn is based on object-oriented programming. Let's start with a quick intro, since I think understanding objects, methods and attributes will help you understand what's going on with the code we'll be using below. Personally, it took me quite some time to figure out exactly what objects are, and why they are so useful. If you have done any programming course in your life, you will have come across them sooner or later, and you'll probably have seen some pretty stupid examples, like this one:

In [ ]:
#let's make a bike:
class Bike:
    def __init__(self,height,wheel_diameter):
        #it has properties, such as a height and a wheel diameter. it's position always starts at 0
        self.height = height
        self.wheel_diameter = wheel_diameter
        self.position = 0
        
    def cycle(self,speed):
        #when you cycle, the bike moves forward by the speed times the wheel diameter 
        #(this is not real cycling, I have no idea about the physics of it)
        self.position = self.wheel_diameter * speed
    
    def get_position(self):
        #the user can call this function to locate the bike at its current position
        return self.position

#okay. let's ride the bike. first, we *instantiate* the object:
my_supercool_bike = Bike(120,70) #let's just take some numbers (that could be centimeters) for the height and wheel

#let's find out the bike's initial position:
initial_pos=my_supercool_bike.get_position()
print("Initial position of the bike: {}".format(initial_pos))

#let's ride the bike
my_supercool_bike.cycle(10) #let's say we cycle at 10km/h

#check the position again
final_pos=my_supercool_bike.get_position()
print("Final position of the bike: {}".format(final_pos))

Easy. Everything clear?! 

Of course not. While this shows the basic structure of defining your own object, it doesn't really tell you why objects are useful, or what they are, really.

Let's start with what the example <b>does</b> show: 
<ul>
<li>Objects are defined with the ```class``` statement
<li>They have <b>methods</b> and <b>attributes</b>. <b>Methods</b> are like functions, and you can call them with ```object.method()```, e.g. ```Bike.cycle()```. <b>Attributes</b> are internal variables that the object stores.
<li>Within the object, the word ```self``` is placed instead of the object name. When the object is instantiated, the variable that is used to instantiate the object replaces ```self``` on the outside. For example, if the method ```cycle``` were to be called inside ```get_position```, we'd use ```self.cycle()```, but once we have our own <b>instance</b> of the object (my_supercool_bike), we call the method by using the object name. The same goes for attributes of the object. 
<li>Every object has an ```__init__``` method that automatically gets called whenever a new object of this type is instantiated. Don't worry about the details of this now. In the case of our bike, this just means that we fill in the attribute values specified by the user, and start at a specific position.
</ul>

What the example fails to show, is why on earth anyone would find this useful. It's just as easy to get the same result using regular variables and functions, like so:

In [ ]:
#define some initial variables
my_wheel_diameter = 70
my_position = 0
my_speed=10
        
def cycle(speed,wheel_diameter,position):
    position = wheel_diameter * speed
    return position
    
#let's find out the bike's initial position:
initial_pos=my_position
print("Initial position of the bike: {}".format(initial_pos))

#cycle and check the position again
final_pos=cycle(my_speed,my_wheel_diameter,my_position)
print("Final position of the bike: {}".format(final_pos))

So, why bother? The reason why classes are useful is exactly their ability to store attributes and methods <b>internally</b>. In the above code, we had to define some global variables in the beginning of our script, and pass them into the function(s) we want to use. Whatever value is returned has to be stored in another variable, or else is forever lost. The function does not remember anything. Objects <b>remember</b> their attribute values. If you want to cycle again, the ```Bike``` object above will remember its current position and wheel diameter, and take it from there. The standalone cycle function won't do that - you have to pass its current position as an argument.

Imagine cooking an elaborate meal, but you only have one pan (a function) and you need to fill anything you make into different plates from your cupboard (variables). Now imagine instead you have this super fancy kitchenaid, which automatically stores all the intermediate results and procedures in some secret hidden place (nobody cares where that is), but you can ask it for a cake or the recipe, anytime you want.

For Machine Learning, this ability of objects to store and remember (so to say: "learn") information is extremely useful. For example, you can instantiate a specific type of model, train its parameters on some data, and then run it on that same data, additional data, etc. You might even throw away all the data altogether, and just keep the model parameters. All you need is the model object, and it remembers where its parameters (e.g., probabilities of observing certain words) are currently at. For some models, you can even update some parameters, using only new, freshly acquired data.

Don't worry if the above introduction is not entirely clear to you. Hopefully you start thinking about python and the world of objects, and it helps you make sense about the way models and methods are used with sklearn. Btw, you probably already know quite a few objects: python strings, lists and dictionaries are all their own type of object, and they have class-specific methods you can call, such as ```.lower()``` for a string, or ```.sort()``` for a list. Psychopy modules also consist largely of objects.

## 2. Train-test split 

Let's start with the schizophrenia dataset. We'll build a model which predicts, for each instance/participant, whether they are schizophrenic or not.

First, we need to split out dataset into a training and test set. We don't have enough data for a proper validation set; we can use cross-validation if we were to tune some hyperparameters. For now, let's keep things simple and use the default value for add-alpha smoothing.

Before you start coding, think about a sensible way to divide the data. How much can we afford to reserve for testing? How much test data do we need? Are the classes very imbalanced? Should we choose our test set randomly, or make sure the distribution of classes is the same for the trainset and testset?

*Take some notes here:*

First, we want to split the data into features and labels seperately, and save each of them in a numpy array. This is helpful for some of the sklearn function we want to use later. As a convention in Machine learning, we use upper case letters (X) to represent matrices and lower case letters (y) for vectors:

In [ ]:
#split the data into features and labels, save them each as numpy.arrays:
X = data_s.drop("label",axis=1).values #X is our feature matrix
y = data_s["label"].values #y is our vector of labels

Sklearn conveniently splits our dataset for us: Use the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">```train_test_split()```</a> function. Save the output of that function in 4 different variables: ```X_train, X_test, y_train, y_test```. Set the percentage of test instances to what you have decided above (it is entered as a float between 0 and 1, 1 being 100%). Our train/test set is pretty small, and therefore vulnerable to random sampling. Therefore, set ```stratify=y```. This ensures the distribution of classes will be about the same in training and testing (is this a realistic assumtion?). Set the ```random_state``` argument of that function =42.

Test the output by using the ```.shape``` attribute of the resulting test and training features/arrays. How many are there in each dataset? Is the feature dimension what you'd expect?

*Note: Many sklearn functions take a ```random_state```. Why do you have to set it? What is it for? Where is there randomness in this function, and where do you think could there be random elements in other Machine Learning classifiers and tools?*

In [ ]:
#Code here.


*Take notes here.*

## 3. A dummy baseline

To see how good our classifier can become, we first need to establish a baseline. In actual research, the baseline is usually the current state-of-the-art. Just to see if we can get anything done at all, we usually also use what is called a <b>dummy baseline</b>. This is the worst performance we can get, without training our model. 

One way of establishing a dummy baseline is to choose a random class for each instance, and measure the accuracy. However, imagine in our dataset, 90% of all participants are actually not scizophrenic, and only 10% are. A completely random classifier would do much worse than one that predicts that every person is not scizophrenic. However, as a diagnosis tool, those two dummy classifiers are equally useless.

Our second dummy classifier therefore selects the most frequent class in the training set for every test person. If this were to give 90% accuracy (because 90% of all participants are in fact in the most frequent class), then we'd hope for our actual model (the Naive Bayes classifier) to get over 90% correct. Failing to beat the dummy baseline can hint at potential problems with our model: Maybe we have too little data, too many features, false assumptions, or just a buggy implementation.

Below, establish two dummy baselines:

1. A random one (using <a href="https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html#numpy.random.choice">```np.random.choice()```</a> to create an output array of the size of the test set (number of test participants), selecting "Patients" or "Control" as random label for each participant)
2. One that chooses the most frequent class in the training set for each test instance (you should be able to figure this one out yourself; *hint: determine the most frequent class by looking at the training labels (use ```np.unique()``` with the argument ```return_counts=True```, then measure what proportion of participants (in percent) fall into that class in the test set*)

In [ ]:
#Code here.


## 4. The Multinomial Naive Bayes Classifier

Let's start building a basic model. We'll use the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB">```MultinomialNB```</a> class, since our output variable is multinomial (technically binomial but we'll ignore that for now). Have a look at the documentation to see what arguments it takes, and how the model is trained.

Afterwards:
- Instantiate the model, using the default parameters (for now)
- Fit the model on the training data, using its ```.fit()``` method
- Predict the test labels using the test features, and save them in an array called ```y_pred``` (for predicted)
- Print the first 5 predicted labels and the first 5 true labels (```y_test```) above one another and compare them. Does the classifier seem to be working?

In [ ]:
# Code here.


Doesn't look too convincing so far. Let's do some proper evaluation to find out how well we are doing. You can use the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html">```accuracy_score```</a> function from ```sklearn.metrics``` to get the accuracy of the classifier. (Alternatively, nb also has a ```.score()``` method.) What accuracy do you get on the training set and test set, respectively? Is there a large difference between training and test accuracy? Why might that be the case? What can we conclude from this?

Compare test accuracy against your dummy baslines. Can we beat that? Reflect on possible causes for poor performance. 

In [ ]:
# Code here.


*Take Notes here.*

Examine the class priors out model has learned (*Hint: chech out the documentation for the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB">```MultinomialNB```</a> class to find which attribute to look at*). Does the result match your expectations (*you may have to convert log probabilities into regular ones with ```np.exp()``` to make sense of it*)?

In [ ]:
#Code here.


Imagine we had collected a dataset with 50% schizophrenic people, but the fraction of people with the condition is much smaller in the actual test population. Let's say, only 10% of all people tested with this classifier (for whom we record some data describing triangles, to find out whether they are schizophrenic) are actually schizophrenic. We know this because of general statistics or the experience of a doctor. Do we have to change our classifier? Can you find out which attribute you would have to change in the MultinomialNB model?

*Take notes here:*

Let's examine the log probabilities the model has learned for words being used by patients versus controls. 
- Which attribute will display the individual feature (word) log probabilities? 
- Find the positions where the log probabilities are highest (> -4.5 in log space, or ca. 0.01 in linear space). *Hint: use <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html">```np.where```</a>*
- Print out the corresopnding words, for each group. *Hint: the ```.columns``` attribute of a dataframe gives you the column names, which should be in the same order as your feature probabilities...*


Do you observe any differences? What could be a problem here?

In [ ]:
# Code here.


*Take Notes here.*

## 5. Feature representation and feature selection 
There are a few things we could do to improve our model, such as tuning the smoothing parameter $\alpha$ in a cross-validation experiment (below). However, the most likely cause for failure is that we have too many features, and too little data. So let's start by working on that.

First of all, you might have noticed that a lot of words with a high likelihood are gereally frequent words, or stop words. The following code creates a new dataset without those stop words. Try to understand what it does.

In [ ]:
#removing stopwords: use a list of known Danish stopwords imported from nltk.corpus
# first, we need to download the stopwords
import nltk
nltk.download('stopwords')
#now we can import them
from nltk.corpus import stopwords
#examine the list of Danish stopwords
stopwords.words('danish')

In [ ]:
#let's remove all the columns that match items in the stopword list:
def remove_stopwords(df):
    for col in df.columns:
        if col in stopwords.words('danish'):
            df.drop(col,inplace=True,axis=1)
    return df

Use the ```remove_stopwords``` function above on our dataset. Check the shape of that new dataset, to see how many of the original features are left after removing stopwords:

In [ ]:
#Code here.


Now divide the data again into training and test set (turn them into matrices/vector X and y again first), fit a new model to this data and evaluate it. Do you observe any changes? Why is that (*Hint: Think about the number of features, the size of our dataset, the assumptions of Naive Bayes, the relevance of stopwords...*)?

In [ ]:
#Code here.


*Take Notes here:*

Check again the log probabilities >-4.5 for both groups. Can you observe some interesting differences now?

In [ ]:
# Code here.


*Take Notes here.*

Another thing we could modify is our original feature representation. For text classification, binary features have sometimes been shown to work better than count features - that is: how often a word occurs matters less than the fact *that* it occurs *at all*. Below, you see some code that will turn one example column of the data frame into a binary array (every value is either 0 - this word occurs, or 1 - this word doesn't occur). Iterate over the columns in ```data_s``` and turn them into binary features, using similar code (or whatever else you prefer). <b>Make sure to not binarize the "label" column with this code(!)</b>

Use ```data_s.head()``` to check whether your data-processing has worked.

In [ ]:
#example for making a Series binary:
example_feature = data_s.columns[0]
data_s[example_feature] = data_s[example_feature].astype('bool').astype('int64')

In [ ]:
#Code here.


Now train and test a new model, using binary features. What do you observe? Does our model get better or worse? Can you think of possible reasons for this?

In [ ]:
#Code here.


*Take Notes here.*

Can you think of any other ways of how to change or select features to improve performance? (You don't actually have to be able to implement it; just take some notes following your linguistic intuition.)

*Take Notes here.*

## 6. Hyperparameter tuning

Next, we want to improve performance by tuning our smoothing parameter $\alpha$. If you've overwritten ```data_s``` to be all binary, you first have to reload the data, because we want to use the original count features. Let's also drop stopwords, since that seemed to work well. Again, save the features and labels in seperate numpy arrays, X and y, and divide them into train and testset, like before:

In [ ]:
#Code here.


Now, we want to find an optimal $\alpha$, using k-fold cross-validation. We'll split our training set into 6 folds (each with 20 instances). We then use a grid search over possible alpha values (within the range 0.1 to 1.0), and see how good our performance gets on the held-out set. We repeat this whole procedure 6 times, using each fold as a held-out set exactly once. In the end, we take the average accuracy as our estimated out-of-sample error for each $\alpga$-value. We choose the best $\alpha$ based on this validation accuracy. Finally, we'll apply the best model to the test set:  we can now fit a new model with our found best $\alpha$ on the entire training set, and measure performance of the resulting model on our testset.

Perform the following steps:
<ol>
<li> Define a range (```np.arange```) of $\alpha$-values to test (save that as ```alphas```).
<li> Get the folds, using sklearn's <a href="http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html">```KFold()```</a> module (save that as ```folds```; this will create an iterable with 6 elements, each of which is a tuple ([train_indeces], [validation_indeces])).
<li> Iterate over the aplphas; for each alpha:
<ol>
<li> Iterate over the folds; for each fold:
<ol>
<li> Fit a multinomial Naive Bayes classifier (with this alpha) to the training folds
<li> Evaluate it on the held-out (validation) fold
<li> Store the validation accuracy (*think: in/outside which loop do you need to define the array to store validation accuracies? what dimensionality does this array have?*)
</ol>
<li> Get the average validation accuracy using this alpha, and store it (*think: in/outside which loop do you need to define the array to store average accuracies per alpha? what dimensionality does this array have?*)
</ol>
<li> Pick the alpha with the highest average validation accuracy
<li> Fit a new model to the full training set (using this alpha)
<li> Evaluate on the test set 
</ol>

How does this change your training/validation/test accuracy? Can you think of possible reasons for this result?

*Hint: this is a challenging coding problem, so if you get stuck, I put some helper functions for you in the bottom of this notebook to use/inspect/get inspired by ;) *

In [ ]:
# Code here.


*Take Notes here:*

## 7. Naive Bayes for Depression [A]

Now it's your turn: Can you apply the techniques you have learned to build a model for depression?

Think about the following before you start:
- Do you want to use 2 classes (depressed/not depressed) or 3 (first-time depressed/chronically depressed/not depressed)?
- How does your choice affect class imbalance? 
- How do you want to split your dataset for training and testing?
- Which dummy baseline will you use?
- Which features/feature representation seems most useful?

*Note: If you are short on time, skip to task 7 now and come back to this later, so we can wrap this up together.*

*Take some notes here:*


In [ ]:
#Code here.

## 8. Findings

Think about your findings. What have they taught you?
- What are the problems with the Naive Bayes approach?
- Do you think it is likely that the Naive Bayes assumptions were met? Why/why not?
- Have you learned anything about depression/schizophrenia?
- Do you think depression/schizophrenia can be predicted from triangles descriptions, generally? What do you think we would need to change to make better predictions?
- What's missing, in general, from this approach?
- How could this approach be used to investigate a specific research question? What additional variables/data would you need for this?

*Take some notes here:*

If you have time, you can read more about the Naive Bayes algorithm and how to improve its performance <a href="https://machinelearningmastery.com/better-naive-bayes/">here</a>.

## 9. Going Further: Finding informative features with Decision Trees [A]

This part is really just additional, advanced material in case you
- are bored
- want to go further (maybe after the workshop is over).

We haven't had time to discuss decision trees, and this little extra bit will not cover them in much detail, either. Basically, they are another kind of model, like Logistic Regression or Naive Bayes. <a href="http://www.speech.zone/classification-and-regression-trees-cart/">This is a cute video</a> introducing Decision Trees, made by my Professor Dr. Simon King in Edinburgh.

The reason why Decision Trees are interesting is that they are <b>interpretable</b>. Importantly, they <b>rank</b> the features by how well they split the two classes in our data - so we can use them to find words which are most different in use between schizophrenic patients and controls! 

Think about how cool this is: We now have a way of automatically determining informative features and selecting them for our classifier. This might also be very helpful if we are doing an explorative study about the language of schizophrenia/depression! This is not quite the same as when we look at log likelihoods in the Naive Bayes classifier, or the coefficients of a Logistic Regression model: These likelihoods/coefficients model each feature in each class independently, and - in the case of logistic regression - possibly on a different scale. The decision tree compares all features against each other, and decides, which best to split the data on. This way, we could select informative features, and subsequently use them in another kind of model.

For now, let's just find the top features in our dataset, so you can get an impression of how this might work. Choose whichever dataset you prefer (schizophrenia and controls/depression and controls). You can leave the stopwords alone for now, since our decision tree should find out by itself whether they are useful or not. Use the same ```X_train, X_test, y_train, y_test``` split as before.

Instantiate an object from the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier>```DecisionTreeClassifier()```</a> class. Use its default parameters.

Fit the model to your training data. Look at the models ```.feature_importances_``` - this gives you a list with importance scores for each feature. Get the non-zero elements of that list (using ```np.where()```). Display the corresponding words of these feature indices (e.g., by indexing into ```data_s.columns```). What do you observe? Which words differentiate well between the two (three) groups in our training set?

Bonus: If you want, you can also evaluate the training and test accuracy with the decision tree. Do you notice any obvious problem? How does the DT beform, compared to our Naive Bayes model? What could be a problem with selecting features this way?

In [ ]:
# Code here.


*Take Notes here.*

## === Helper Functions ===

In [ ]:
def get_cross_validation_acc(folds,alpha,X_train,y_train):
    """
    A function that calculates the average held-out validation accuracy (with Naive Bayes) for a set of folds, 
    using each fold as validation set once. 
    :param folds: KFold() iterable object
    :param alpha: alpha value to be used in the Naive Bayes classifier
    :param X_train: training feature array
    :param y_train: training feature labels
    """
    val_accs = np.zeros((len(folds),))
    j=0
    for train_idx, val_idx in folds:
        nb = MultinomialNB(alpha=alpha)
        nb.fit(X_train[train_idx], y_train[train_idx])
        y_pred = nb.predict(X_train[val_idx])
        val_accs[j] = accuracy_score(y_train[val_idx], y_pred)
        j+=1
    return np.mean(val_accs)

def get_best_alpha_and_acc(alphas,accs):
    """
    Find the best alpha value and its corresponding validation accuracy, 
    given an array of alphas and corresponding validation accuracies.
    """
    highest_acc = np.max(accs)
    index_of_highest = np.where(accs==highest_acc)[0][0]
    best_alpha = alphas[index_of_highest]
    return best_alpha, highest_acc